# Sending JJY signal with AD9833 via FTDI
- https://en.wikipedia.org/wiki/JJY  
- https://www.analog.com/media/en/technical-documentation/data-sheets/ad9833.pdf

In [1]:
import os, sys

def gen_relative_path(target_path): 
    
    def del_same_parents(target_path, current_path):
        if len(target_path) * len(current_path) > 0:
            if target_path[0] == current_path[0]:
                target_path.pop(0)
                current_path.pop(0)
                del_same_parents(target_path, current_path)
            
    current_path = os.getcwd().replace('\\', '/').split('/')
    target_path = target_path.replace('\\', '/').split('/')
    del_same_parents(target_path, current_path)
    
    return ['..'] * len(current_path) + target_path


def append_source_relative_path(source_relative_path):
    sys.path.append(os.sep.join(source_relative_path))

In [2]:
# paths = ['']
# paths = [gen_relative_path(p) for p in paths]
# print(paths)

In [3]:
paths = [['..', '..', '..', '..', '..', '..', '待處理', 'ORM'],
         ['..', '..', '..', '..', '..', '..', '待處理', 'USB', 'Universal Serial Bus', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', 'Bridges', 'bitbucket', 'github', 'codes'],
         ['..', '..', '..', '..', '..', '..', '待處理', 'Signal Generators', 'bitbucket', 'github', 'codes'],
         ['..', 'codes']]

for path in paths:
    append_source_relative_path(path)

In [4]:
%pylab inline 

from signal_generators import adapters, register, interfaces, shift_register
from signal_generators.ad983x.ad9833 import ad9833, modulators
from radio_clock.jjy import *

Populating the interactive namespace from numpy and matplotlib


## Debug mode?

In [5]:
adapters.SPI.DEBUG_MODE = False  # whether to show SPI written data.
ad9833.AD9833.DEBUG_MODE = False        # whether to dump registers. 

## Generators and Modulators

In [7]:
with_hardware_device = True

if with_hardware_device:
    _clk = adapters.Pin.get_Ftdi_pin(pin_id = 4)
    _data = adapters.Pin.get_Ftdi_pin(pin_id = 1)

    _ss = adapters.Pin.get_Ftdi_pin(pin_id = 3)
    _spi = shift_register.ShiftRegister(stb_pin = _ss, clk_pin = _clk, data_pin = _data, polarity = 1) 
else:
    _spi = _ss = None  # using None for testing without actual hardware device.

In [8]:
ook = modulators.OOK(_spi, _ss, freq = 4e4)

In [ ]:
ook.enable_output(True)

In [ ]:
ook.enable_output(False)

In [ ]:
year, month, month_day, hour, minute, second, week_day, year_day = Encoder.get_datetime_elements()
year, month, month_day, hour, minute, second, week_day, year_day    

## Sending JJY signal

In [10]:
import time 
for i in range(2):
    t = time.time()
    print(time.localtime(t))
    
    jjy_sequence = Encoder.get_symbols_ook_sequence(t)
    ook.send_sequence(jjy_sequence)
    
#     time.sleep(5)

time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=14, tm_min=14, tm_sec=21, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=14, tm_min=15, tm_sec=21, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=14, tm_min=16, tm_sec=21, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=14, tm_min=17, tm_sec=21, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=14, tm_min=18, tm_sec=21, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=14, tm_min=19, tm_sec=21, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=14, tm_min=20, tm_sec=21, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=14, tm_min=21, tm_sec=21, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, t

time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=15, tm_min=22, tm_sec=16, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=15, tm_min=23, tm_sec=15, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=15, tm_min=24, tm_sec=15, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=15, tm_min=25, tm_sec=15, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=15, tm_min=26, tm_sec=15, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=15, tm_min=27, tm_sec=15, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=15, tm_min=28, tm_sec=15, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, tm_mon=4, tm_mday=3, tm_hour=15, tm_min=29, tm_sec=15, tm_wday=4, tm_yday=94, tm_isdst=0)
time.struct_time(tm_year=2020, t